Experiments with Mathematica Engine

In [ ]:
RewFormat`RewFormatImport[filename_String, options___] :=
 
 Module[{strm, head, data},
  strm = OpenRead[filename];
  head = ReadList[strm, "String", 14];
  data = Flatten[Partition[ReadList[strm, Record, RecordSeparators -> {"\n"}], 15]];
  real = Transpose[Table[Interpreter["Number"][StringSplit[x, ","]], {x, data}]];
  freq = real[[1]];
  gain = real[[2]];
  phase= real[[3]];
  Close[strm];
  (* the converter returns a list of rules,
   in the form of ElementName \[Rule] ElementValue *)
  {
   "Header" -> head,
   "Freq" -> freq,
   "Gain"-> gain,
   "Phase" -> phase
   }
  ]
  
ImportExport`RegisterImport["REW", RewFormat`RewFormatImport]

In [ ]:
L = Import["../datas/inroom/m001/L.txt", {"REW"}];
R = Import["../datas/inroom/m001/R.txt", {"REW"}];

In [ ]:
Tau = 360;
Unwrap[a_] := a - Tau Prepend[Accumulate[Round[Differences[a]/Tau]], 0];

Lfreq = L[[2]][[2]];
Lgain = L[[3]][[2]];
Lphase = Unwrap[L[[4]][[2]]*Pi/180];
LPointsGain = Transpose@{Lfreq, Lgain};
LPointsPhase = Transpose@{Lfreq, Lphase};
Rfreq = R[[2]][[2]];
Rgain = R[[3]][[2]];
Rphase = Unwrap[R[[4]][[2]]*Pi/180];
RPointsGain = Transpose@{Rfreq, Rgain};
RPointsPhase = Transpose@{Rfreq, Rphase};

In [ ]:
maxFreq = 300;
ListLinePlot[
    {Select[LPointsGain, #[[1]]<maxFreq&], Select[RPointsGain, #[[1]]<maxFreq&]},
    AxesOrigin -> {20, 30},
    FrameLabel -> {"Frequenzy (Hz)", "dB SPL"},
    PlotLabel -> "(Gain) Left v.s. Right for test1",
    PlotLegends -> {"Left", "Right"}, 
    ScalingFunctions -> {"Log", None},
    ImageSize -> Large,
    PlotTheme -> "Detailed"]

In [ ]:
maxFreq = 300;
ListLinePlot[
    {Select[LPointsPhase, #[[1]]<maxFreq&], Select[RPointsPhase, #[[1]]<maxFreq&]}, 
    FrameLabel -> {"Frequenzy (Hz)", "Phase (Deg)"},
    PlotLabel -> "(Phase) Left v.s. Right for test1",
    PlotLegends -> {"Left", "Right"}, 
    ScalingFunctions -> {"Log", None},
    ImageSize -> Large,
    PlotTheme -> "Detailed"]

In [ ]:
maxFreq = 300;
LGroupDelay = ListConvolve[{1, -1}, Select[LPointsPhase, #[[1]]<maxFreq&][[All, 2]]];
RGroupDelay = ListConvolve[{1, -1}, Select[RPointsPhase, #[[1]]<maxFreq&][[All, 2]]];


In [ ]:
ListLinePlot[
    {LGroupDelay, RGroupDelay}, 
    FrameLabel -> {"Time (ms)", "Deg/s"},
    PlotLabel -> "(Group Delay) Left v.s. Right for test1",
    PlotLegends -> {"Left", "Right"}, 
    ImageSize -> Large,
    PlotTheme -> "Detailed"]

In [ ]:
Lz = Map[Complex[#1[[1]], #1[[2]]]&, LPointsPhase];
Rz = Map[Complex[#1[[1]], #1[[2]]]&, RPointsPhase];

In [ ]:
pLz = Arg[InverseFourier[Lz]];
pRz = Arg[InverseFourier[Rz]];
ListPlot[{pLz, pRz}]